# SQLite itegration

If you want to process your sensor data and store it for later, you can use the sqlite integration. Gensor's `Timeseries` and `Dataset` come with a `.to_sql()` method which is uses `pandas.Series.to_sql()` method under the hood to save the data in a SQLite database. 

It is a simple implementation, where each timeseries is stored in a separate schema (database table) which is named in the following pattern: `f"{location}_{sensor}_{variable}_{unit}".lower()`. There is a double check on duplicates. First, when you create a `Dataset`, duplicates are nicely handled by merging timeseries from the same location, sensor and of the same variable and unit. Secondly the `Timeseries.to_sql()` method is designed to ignore conflicts, so only new records are inserted into the database if you attempt to run the same commend twice.

### Load test data

In [ ]:
import gensor as gs
from gensor.testdata import all_paths, pb02a_plain

from gensor import read_from_csv

pattern = r'[A-Za-z]{2}\d{2}[A-Za-z]{1}|Barodiver'

ds = read_from_csv(path=all_paths, 
                    file_format='vanessen', 
                    location_pattern=pattern)


ds2 = read_from_csv(path=pb02a_plain, 
                    file_format='plain', 
                    location='PB02A',
                    sensor='AV336')

ds.add(ds2)


### Create `DatabaseConnection`

Both saving and loading data from sqlite require a `DatabaseConnection` object to be passed as attribute. You can just instanciate it with empty parentheses to create a new database in the current working directory, or specify the path and name of the database.

If you have an existing Gensor database, you can use `DatabaseConnection.get_tables()` to see if there already are some tables in the database that you want to use.

In [ ]:
db = gs.db.DatabaseConnection()
db.get_tables()

Loading the dataset to the database is straightforward. You just need to call `.to_sql()` on the dataset instance and check the tables again to see that now there are a few.

In [ ]:
ds.to_sql(db)
db.get_tables()

In [ ]:
from gensor import read_from_sql, Dataset

# Issue: the unit name is case sensitive (shouldn't be)
new_ds: Dataset = read_from_sql(db, True)
new_ds.plot()